In [1]:
%matplotlib inline
from Utils.config import generalConfig, structDqnConfig, model_dir
import numpy as np
np.random.seed(generalConfig["global_seed"])
from keras.models import Graph
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Dropout, Activation, Masking
from keras.layers.embeddings import Embedding
from keras.optimizers import RMSprop
from keras.layers.core import TimeDistributedMerge, TimeDistributedDense
from keras.layers import containers
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import pickle as pkl
from sklearn.neighbors import NearestNeighbors
from keras import backend as K

Using Theano backend.


In [2]:
graph = Graph()

usr_size = 10
sys_size = 10
sys_embed = 10
usr_embed = 30
embed_size = sys_embed + usr_embed + 1

graph.add_input(name='usr', input_shape=(None, usr_size))
graph.add_input(name='sys', input_shape=(None, sys_size))
graph.add_input(name='cmp', input_shape=(None, 1))

# add masking
graph.add_node(Masking(mask_value=0.0, input_shape=(None, usr_size)), name="usr_mask", input="usr")
graph.add_node(Masking(mask_value=0.0, input_shape=(None, sys_size)), name="sys_mask", input="sys")

graph.add_node(TimeDistributedDense(sys_embed, input_dim=sys_size, init='he_normal'), name="sys_embed", input="sys_mask")
graph.add_node(TimeDistributedDense(usr_embed, input_dim=usr_size), name="usr_embed", input="usr_mask")


graph.add_node(Me(mask_value=0.0, input_shape=(None, embed_size)),
               inputs=["sys_embed", "usr_embed", "cmp"], name="turn_embed", merge_mode='concat')

loss = {}

#shared_model.add(Masking(mask_value=0.0, input_shape=(None, embed_size)))
shared_model = containers.Sequential()
shared_model.add(LSTM(256, input_dim=embed_size, return_sequences=False))
shared_model.add(Dropout(0.3))

graph.add_node(shared_model, name="recurrent_layers", input="turn_embed")
#graph.add_node(shared_model, name="recurrent_layers", inputs=["sys_embed", "usr_embed", "cmp"], merge_mode='concat')

# policies
policy_action_num = {"verbal":32, "computer": 3}

for p_name in ["verbal", "computer"]:
    graph.add_node(Dense(structDqnConfig["l1-"+p_name], activation='tanh'), name="l1-"+p_name, input="recurrent_layers")
    graph.add_node(Dropout(0.3), name="l1dp-"+p_name, input="l1-"+p_name)
    graph.add_node(Dense(policy_action_num[p_name], activation='linear'), name=p_name, input='l1dp-'+p_name, create_output=True)
    loss[p_name] = "mse"

opt = RMSprop(clipvalue=1.0)
graph.compile(optimizer=opt, loss=loss)

In [3]:
print graph.summary()

--------------------------------------------------------------------------------
Layer (name)                  Output Shape                  Param #             
--------------------------------------------------------------------------------
Layer (usr)                   (None, None, 10)              0                   
Layer (sys)                   (None, None, 10)              0                   
Layer (cmp)                   (None, None, 1)               0                   
Masking (usr_mask)            (None, None, 10)              0                   
Masking (sys_mask)            (None, None, 10)              0                   
TimeDistributedDense (sys_embe(None, None, 10)              110                 
TimeDistributedDense (usr_embe(None, None, 30)              330                 
Masking (turn_embed)          (None, None, 41)              0                   
Sequential (recurrent_layers) (None, 256)                   305152              
Dense (l1-verbal)           

In [6]:
# make bias non-zero
sys_w = graph.nodes["sys_embed"].get_weights()
sys_w[1] = np.zeros(sys_embed)
graph.nodes["sys_embed"].set_weights(sys_w)

# user weights
usr_w = graph.nodes["usr_embed"].get_weights()
usr_w[1] = np.zeros(usr_embed)
graph.nodes["usr_embed"].set_weights(usr_w)


In [18]:
turn_embed_output = K.function([graph.inputs[i].input for i in graph.input_order],
                              [graph.nodes['recurrent_layers'].get_output(train=False)])
usr_embed_output = K.function([graph.inputs['usr'].input],
                              [graph.nodes['usr_embed'].get_output_mask(train=False)])

In [11]:
# fake input
sys = np.zeros((1, 5, sys_size))
usr = np.zeros((1, 5, usr_size))
com = np.zeros((1, 5, 1))
sys[0, [2,3], :] = 1.0
usr[0, [2,4], :] = 1.0
com[0, [3], :] = 1.0

input_dict = {'usr':usr, 'sys':sys, 'cmp':com}
input_list = [usr, sys, com]

In [18]:
print graph.predict(input_dict)

{'computer': array([[-0.11927572,  0.20427363,  0.02865927]]), 'verbal': array([[ 0.00402713, -0.11726081,  0.07506087, -0.34203163,  0.11910778,
         0.10639571, -0.04797643, -0.37948903,  0.17089853, -0.14354943,
         0.07060702,  0.05614262,  0.11790424,  0.04424603,  0.08680534,
        -0.0780632 , -0.18311547,  0.02878226,  0.11700052, -0.02067673,
        -0.31823581,  0.2506201 ,  0.31887996,  0.19409446, -0.2112639 ,
         0.10458054, -0.07540109,  0.31896442,  0.12354691, -0.06275554,
         0.23527353,  0.08727904]])}


In [19]:
output = turn_embed_output(input_list)[0]
print output

[[0 0 1 1 1]]


In [14]:
print usr_embed_output([usr])[0]

[[0 0 1 0 1]]


In [15]:
usr

array([[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.]]])